# Running Models with ONNX

## Running A Dummy Input

But before verifying the model’s output with ONNX Runtime, we will check the ONNX model with ONNX’s API. 

First, onnx.load("super_resolution.onnx") will load the saved model and will output a onnx.ModelProto structure (a top-level file/container format for bundling a ML model. 

For more information onnx.proto documentation.). Then, onnx.checker.check_model(onnx_model) will verify the model’s structure and confirm that the model has a valid schema. 

The validity of the ONNX graph is verified by checking the model’s version, the graph’s structure, as well as the nodes and their inputs and outputs.

In [6]:
! pip install -q onnx
! pip install -q onnxruntime

     |████████████████████████████████| 3.7MB 99kB/s 


In [0]:
import onnx
import onnxruntime

In [0]:
from PIL import Image
import torchvision.transforms as transforms

In [0]:
import numpy as np

In [0]:
import torch

In [0]:
onnx_model = onnx.load("super_resolution.onnx")
onnx.checker.check_model(onnx_model)

In [0]:
ort_session = onnxruntime.InferenceSession("super_resolution.onnx")

In [0]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


In [0]:
batch_size = 1

In [0]:
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)

In [0]:
# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

In [17]:
print(ort_outs[0])

[[[[ 0.2905327   0.09409931 -0.25207695 ...  0.19432266  0.16818433
     0.3392768 ]
   [ 0.5156717   0.22253723 -0.14361352 ...  0.43314967  0.39443794
     0.47478086]
   [ 0.598842    0.61774474  0.44808784 ...  0.7250732   0.97829145
     0.9215888 ]
   ...
   [ 0.60323215  0.6122704   0.52220565 ...  0.0055224  -0.1838183
    -0.22857508]
   [ 0.42479503  0.35224244  0.40549043 ...  0.00487928 -0.04999856
    -0.0598807 ]
   [ 0.24426788  0.17307022  0.25811365 ...  0.00270854  0.0311784
     0.02342708]]]]


We should see that the output of PyTorch and ONNX Runtime runs match numerically with the given precision (rtol=1e-03 and atol=1e-05). As a side-note, if they do not match then there is an issue in the ONNX exporter, so please contact us in that case.

## Running the model on an image using ONNX Runtime

So far we have exported a model from PyTorch and shown how to load it and run it in ONNX Runtime with a dummy tensor as an input.

For this tutorial, we will use a famous cat image used widely which looks like below

First, let’s load the image, pre-process it using standard PIL python library. Note that this preprocessing is the standard practice of processing data for training/testing neural networks.

We first resize the image to fit the size of the model’s input (224x224). Then we split the image into its Y, Cb, and Cr components. 

These components represent a greyscale image (Y), and the blue-difference (Cb) and red-difference (Cr) chroma components. 

The Y component being more sensitive to the human eye, we are interested in this component which we will be transforming. 

After extracting the Y component, we convert it to a tensor which will be the input of our model.

In [0]:
img = Image.open("/content/cat.jpg")

In [0]:
resize = transforms.Resize([224, 224])

In [0]:
img = resize(img)

In [0]:
img_ycbcr = img.convert('YCbCr')
img_y, img_cb, img_cr = img_ycbcr.split()

In [0]:
to_tensor = transforms.ToTensor()

In [0]:
img_y = to_tensor(img_y)

In [36]:
print(img_y.shape)

torch.Size([1, 224, 224])


In [0]:
# add batch_size dim
img_y.unsqueeze_(0)

In [40]:
print(img_y.shape)

torch.Size([1, 1, 224, 224])


Now, as a next step, let’s take the tensor representing the greyscale resized cat image and run the super-resolution model in ONNX Runtime as explained previously.

In [0]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img_y)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y = ort_outs[0]

In [0]:
img_out_y = Image.fromarray(np.uint8((img_out_y[0] * 255.0).clip(0, 255)[0]), mode='L')

# get the output image follow post-processing step from PyTorch implementation
final_img = Image.merge(
    "YCbCr", [
        img_out_y,
        img_cb.resize(img_out_y.size, Image.BICUBIC),
        img_cr.resize(img_out_y.size, Image.BICUBIC),
    ]).convert("RGB")

In [0]:
final_img.save("cat_superres_with_ort.jpg")